In [ ]:
import os
import json
import jsonlines
import time
import pandas as pd

import wandb
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)
system_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly. The assistant will attempt to give a response that is concise but ensures that all the key points are included when relevant."

In [ ]:
def complete_chat(message, temperature, seed, model, system_prompt=system_prompt, client=client):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system", 
                "content": system_prompt
            }, 
            {
                "role": "user",
                "content": message,
            },
        ],
        model=model,
        temperature=temperature,
        seed=seed,
    )
    # Extract the response from the chat completion
    message = chat_completion.choices[0].message.content
    return message

# Summarization of a big document

In [ ]:
# Read a jsonl file with jsonl library 
def read_jsonl(filepath):
    with jsonlines.open(filepath) as reader:
        posts = []
        for obj in reader:
            posts.append(json.dumps(obj))

    return '\n\n'.join(posts)

blog_posts = read_jsonl('data/blog_trim.jsonl')
blog_posts[:100]

In [ ]:
def run_summarize_experiment(data, seed, temperature, model, iterations=10):
    entries = []
    for i in range(iterations):
        print(f'Iteration {i+1}')
        prompt = "Summarize the following: " + data
        response = complete_chat(prompt, 
                                 seed=seed, 
                                 temperature=temperature, 
                                 model=model if i > 0 else 'gpt-4o')
        entry = {
            "iteration": i,
            "prompt": prompt,
            "seed": seed,
            "model": model,
            "temperature": temperature,
            "response": response,
            "length": len(response),
        }
        #Log as table in wandb
        wandb.log(entry)

        print(f'* data: {len(data)}')
        print(f'* len: {entry["length"]}')
        print('')

        data = response
        entries.append(entry)

    return pd.DataFrame(entries)

In [ ]:
seed = 42
iterations = 50

def make_run_name(config):
    return f"run-{config['model']}_{config['seed']}_{config['temperature']}"

dfs = []
for temperature in [0.0, 0.5, 1.0, 2.0]:
    for model in ['gpt-4o', 'gpt-3.5-turbo']:
        config = {
            "seed": seed,
            "temperature": temperature,
            "model": model,
            "iterations": iterations,
        }

        wandb.init(project="llm_fixed_point",
                   name=make_run_name(config),
                   config=config)

        data = blog_posts
        df = run_summarize_experiment(data, **config)
        dfs.append((config, df))

        wandb.finish()

        # wait 30 seconds to avoid rate limiting
        time.sleep(30)

In [ ]:
df = pd.concat([x for _, x in dfs])
df.head()

df.to_csv('data/summarize_blog_posts.csv', index=False)

In [ ]:
read_df = pd.read_csv('data/summarize_blog_posts.csv')

def compare(df, model, temperature):
    df = df[(df.model == model) & (df.temperature == temperature)]['response']
    print(df.iloc[0])
    print('#' * 100)
    print(df.iloc[-1])

compare(df=read_df, model='gpt-4o', temperature=0.5)

# Summarize List of Items

In [ ]:
def run_list_summarize_experiment(data, seed, temperature, model, prompt, iterations=10):
    entries = []
    for i in range(iterations):
        print(f'Iteration {i+1}')
        prompt = prompt + data
        response = complete_chat(prompt, 
                                 seed=seed, 
                                 temperature=temperature, 
                                 model=model if i > 0 else 'gpt-4o')
        entry = {
            "iteration": i,
            "prompt": prompt,
            "seed": seed,
            "model": model,
            "temperature": temperature,
            "response": response,
            "length": len(response),
            "lines": len(response.split('\n')),
        }
        #Log as table in wandb
        wandb.log(entry)

        print(f'* data: {len(data)}')
        print(f'* len: {entry["length"]}')
        print(f'* lines: {entry["lines"]}')
        print('')

        data = response
        entries.append(entry)

    return pd.DataFrame(entries)

In [ ]:
list_prompt = "Generate a list of exactly 100 random facts.  Don't add any additional text, just output the list."

data = complete_chat(
    list_prompt, seed=42, temperature=1.0, model="gpt-4o"
)

print(len(data.split('\n')))
print(data)

In [ ]:
seed = 42
iterations = 50

def make_run_name(config):
    return f"{config['prompt'].split('')[0]}-run-{config['model']}_{config['seed']}_{config['temperature']}"

dfs = []
for temperature in [0.0, 0.5, 1.0, 2.0]:
    for model in ['gpt-4o', 'gpt-3.5-turbo']:
        for prompt in ['Summarize the following: ', 'Rephrase the following: ']:
            config = {
                "prompt": prompt,
                "seed": seed,
                "temperature": temperature,
                "model": model,
                "iterations": iterations,
            }
            wandb.init(project="llm_fixed_point",
                       name=make_run_name(config),
                       config=config)

            df = run_list_summarize_experiment(data, prompt="", **config)
            dfs.append((config, df))

            wandb.finish()
            break

In [ ]:
df = pd.concat([x for _, x in dfs])
df.to_csv('data/summarize_list.csv', index=False)
df.head()

In [ ]:
read_df = pd.read_csv('data/summarize_list.csv')

def compare(df, model, temperature):
    df = df[(df.model == model) & (df.temperature == temperature)]['response']
    print(df.iloc[0])
    print('#' * 100)
    print(df.iloc[-1])

compare(df=read_df, model='gpt-4o', temperature=0.5)